In [2]:
import pandas as pd

df=pd.read_csv("Train.csv")

In [3]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df.drop(columns=["Item_Identifier","Outlet_Establishment_Year"],inplace=True,axis=1)

In [5]:
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.7052


In [6]:
df.shape

(8523, 10)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Item_Weight           7060 non-null   float64
 1   Item_Fat_Content      8523 non-null   object 
 2   Item_Visibility       8523 non-null   float64
 3   Item_Type             8523 non-null   object 
 4   Item_MRP              8523 non-null   float64
 5   Outlet_Identifier     8523 non-null   object 
 6   Outlet_Size           6113 non-null   object 
 7   Outlet_Location_Type  8523 non-null   object 
 8   Outlet_Type           8523 non-null   object 
 9   Item_Outlet_Sales     8523 non-null   float64
dtypes: float64(4), object(6)
memory usage: 666.0+ KB


In [8]:
# Calculate the mean of non-zero values in the Item_Visibility column
mean_visibility = df[df['Item_Visibility'] != 0]['Item_Visibility'].mean()

# Replace 0 values with the calculated mean
df['Item_Visibility'] = df['Item_Visibility'].replace(0, mean_visibility)

In [9]:
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.070482,Fruits and Vegetables,182.0950,OUT010,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.070482,Household,53.8614,OUT013,High,Tier 3,Supermarket Type1,994.7052


In [10]:
df.to_csv("modifiedSPS.csv")

In [11]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from src.exception import CustomException
from sklearn.preprocessing import StandardScaler
from src.logger import logging
import sys

In [12]:
try:
    numerical_columns=['Item_Weight', 'Item_Visibility', 'Item_MRP',]
    categorical_columns=['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size',
       'Outlet_Location_Type', 'Outlet_Type']
    num_pipeline=Pipeline(

                steps=[
                    ("imputer",SimpleImputer(strategy="median")),
                    ("scaler",StandardScaler())
                ]
            )

    cat_pipeline=Pipeline(

                steps=[
                    ("imputer",SimpleImputer(strategy="most_frequent")),
                    ("one_hot_encoder",OneHotEncoder(handle_unknown='ignore')),
                    ("scaler",StandardScaler(with_mean=False))


                ]
            )
            

    preprocessor=ColumnTransformer(

                [
                ("num_pipeline",num_pipeline,numerical_columns),
                ("cat_pipeline",cat_pipeline,categorical_columns)
                 ]
            )
except Exception as e:
    raise CustomException(e,sys)

In [13]:
from sklearn.model_selection import train_test_split
train_df,test_df=train_test_split(df,test_size=0.4,random_state=42)

In [14]:
train_df.shape,test_df.shape

((5113, 10), (3410, 10))

In [15]:
target_column_name="Item_Outlet_Sales"

In [16]:
input_feature_train_df=train_df.drop(columns=[target_column_name],axis=1)
target_feature_train_df=train_df[target_column_name]

input_feature_test_df=test_df.drop(columns=[target_column_name],axis=1)
target_feature_test_df=test_df[target_column_name]

In [17]:
input_feature_train_df.shape,target_feature_train_df.shape,input_feature_test_df.shape,target_feature_test_df.shape

((5113, 9), (5113,), (3410, 9), (3410,))

In [18]:
input_feature_train_arr=preprocessor.fit_transform(input_feature_train_df)
input_feature_test_arr=preprocessor.transform(input_feature_test_df)

In [19]:
import numpy as np

In [20]:
train_arr = np.hstack((input_feature_train_arr.toarray(), target_feature_train_df.values.reshape(-1, 1)))
test_arr = np.hstack((input_feature_test_arr.toarray(), target_feature_test_df.values.reshape(-1, 1)))

In [21]:
pd.DataFrame(train_arr)

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,-1.171540,-1.151980,-0.923603,0.000000,2.037702,0.000000,0.000000,0.0,3.874172,0.0,...,0.000000,2.227481,2.239607,0.000000,0.000000,0.000000,2.101943,0.000000,0.0,416.1250
1,1.375746,-0.359091,0.796916,0.000000,2.037702,0.000000,0.000000,0.0,0.000000,0.0,...,2.051363,0.000000,0.000000,0.000000,2.044677,0.000000,0.000000,3.148598,0.0,2109.2544
2,-0.019056,1.064751,-1.582697,0.000000,2.037702,0.000000,0.000000,0.0,0.000000,0.0,...,2.051363,0.000000,0.000000,2.128396,0.000000,0.000000,2.101943,0.000000,0.0,703.0848
3,1.789458,1.510858,-1.677045,0.000000,2.037702,0.000000,0.000000,0.0,0.000000,0.0,...,2.051363,0.000000,0.000000,2.128396,0.000000,0.000000,2.101943,0.000000,0.0,476.0470
4,-0.030876,2.490544,-1.599127,0.000000,0.000000,2.119874,0.000000,0.0,0.000000,0.0,...,0.000000,2.227481,2.239607,0.000000,0.000000,3.042434,0.000000,0.000000,0.0,42.6112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5108,-0.835842,4.459255,-0.053053,0.000000,0.000000,2.119874,0.000000,0.0,0.000000,0.0,...,2.051363,0.000000,0.000000,0.000000,2.044677,3.042434,0.000000,0.000000,0.0,280.9676
5109,0.631064,0.973933,-1.064684,0.000000,2.037702,0.000000,0.000000,0.0,0.000000,0.0,...,2.051363,0.000000,0.000000,2.128396,0.000000,0.000000,2.101943,0.000000,0.0,1301.6390
5110,1.103878,-1.062921,1.510582,0.000000,2.037702,0.000000,0.000000,0.0,0.000000,0.0,...,2.051363,0.000000,0.000000,2.128396,0.000000,0.000000,2.101943,0.000000,0.0,6145.3340
5111,1.753997,-0.331473,-0.391297,0.000000,0.000000,0.000000,8.373128,0.0,0.000000,0.0,...,2.051363,0.000000,0.000000,2.128396,0.000000,0.000000,2.101943,0.000000,0.0,1649.8524


In [22]:
train_arr.shape,test_arr.shape

((5113, 45), (3410, 45))